In [2]:
import pandas as pd

In [9]:
# Import data
snapshot_1 = pd.read_csv('data/snapshot_1.csv')
snapshot_2 = pd.read_csv('data/snapshot_2.csv')

# rename snapshot_2's columns to match snapshot_1's assuming they have the same column data in the same column order
snapshot_2.columns = snapshot_1.columns

In [16]:
# cleans the sku's to have uppercase letters for the 'SKU' part of their item code
snapshot_2['sku'] = snapshot_2['sku'].str.upper()
# adds a dash between the first capture group ('SKU') and second capture group (###)
snapshot_2['sku'] = snapshot_2['sku'].str.replace(r'(SKU)(\d+)', r'\1-\2', regex=True)

In [7]:
snapshot_1

,sku,name,quantity,location,last_counted
0,SKU-001,Widget A,150,Warehouse A,2024-01-08
1,SKU-002,Widget B,75,Warehouse A,2024-01-08
2,SKU-003,Gadget Pro,200,Warehouse B,2024-01-08
3,SKU-004,Gadget Lite,50,Warehouse A,2024-01-08
4,SKU-005,Connector Cable 6ft,500,Warehouse C,2024-01-08
...,...,...,...,...,...
70,SKU-071,USB Microphone,30,Warehouse A,2024-01-08
71,SKU-072,XLR Microphone,20,Warehouse A,2024-01-08
72,SKU-073,Pop Filter,100,Warehouse B,2024-01-08
73,SKU-074,Boom Arm,45,Warehouse B,2024-01-08


In [21]:
snapshot_2['sku'].values

<StringArray>
['SKU-001', 'SKU-002', 'SKU-003', 'SKU-004', 'SKU-005', 'SKU-006', 'SKU-007',
 'SKU-008', 'SKU-009', 'SKU-010', 'SKU-011', 'SKU-012', 'SKU-013', 'SKU-014',
 'SKU-015', 'SKU-016', 'SKU-017', 'SKU-018', 'SKU-019', 'SKU-020', 'SKU-021',
 'SKU-022', 'SKU-023', 'SKU-024', 'SKU-027', 'SKU-028', 'SKU-029', 'SKU-030',
 'SKU-031', 'SKU-032', 'SKU-033', 'SKU-034', 'SKU-035', 'SKU-036', 'SKU-037',
 'SKU-038', 'SKU-039', 'SKU-040', 'SKU-041', 'SKU-042', 'SKU-043', 'SKU-044',
 'SKU-045', 'SKU-046', 'SKU-047', 'SKU-048', 'SKU-049', 'SKU-050', 'SKU-051',
 'SKU-052', 'SKU-053', 'SKU-054', 'SKU-045', 'SKU-055', 'SKU-056', 'SKU-057',
 'SKU-058', 'SKU-059', 'SKU-060', 'SKU-061', 'SKU-062', 'SKU-063', 'SKU-064',
 'SKU-065', 'SKU-066', 'SKU-067', 'SKU-068', 'SKU-069', 'SKU-070', 'SKU-071',
 'SKU-072', 'SKU-073', 'SKU-074', 'SKU-075', 'SKU-076', 'SKU-077', 'SKU-078',
 'SKU-079', 'SKU-080']
Length: 79, dtype: str

Identifies:

Items present in both snapshots (and whether quantities changed)

Items only in snapshot 1 (removed/sold out)

Items only in snapshot 2 (newly added)

Any data quality issues worth flagging


Data issues:

    column names change between files
        name to product_name
        quantity to qty
        location to warehouse
        last_counted to updated_at
    sku
        format (missing dash)
        capital letters